This is a momentum based portfolio using the tickers from the Russell 2000 ETF (IWM)

In [1]:
#import packages
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
from statistics import mean
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read in tickers
stocks = pd.read_csv('russ_tickers.csv')

In [3]:
#this is a common Sandbox API for the IEX Cloud
#It will pull in stale data
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [4]:
#create columns for the data frame
columns = [
                'Ticker', 'Company Name',  
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'Score','Price','Number of Shares to Buy','Position Value'
                ]

In [5]:
df = pd.DataFrame(columns = columns)
df

,Ticker,Company Name,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Score,Price,Number of Shares to Buy,Position Value


In [6]:
#This function will break the list of tickers into smaller lists
#The API works quicker when this is are broken into smaller pieces
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [7]:
#create the smaller lists of tickers
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [ ]:
#retreive the data
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
                                                   pd.Series([symbol, data[symbol]['quote']['companyName'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A',
                                                   data[symbol]['quote']['latestPrice'],
                                                    'N/A',
                                                    'N/A' 
                                                   ],
                                                  index = columns), 
                                        ignore_index = True)
        
df.head()

In [ ]:
#calculate and populate the percentile return columns
periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in df.index:
    for period in periods:
        df.loc[row, f'{period} Return Percentile'] = stats.percentileofscore(df[f'{period} Price Return'],
        df.loc[row, f'{period} Price Return'])/100


df.head()

In [ ]:
#create a score for each stock
for row in df.index:
    percentiles = []
    for period in periods:
        percentiles.append(df.loc[row, f'{period} Return Percentile'])
    df.loc[row, 'Score'] = mean(percentiles)

df.head()

In [ ]:
#sort the stocks by score
df.sort_values(by = 'Score', ascending = False, inplace = True)
#create a new dataframe with just the top 100 stocks
df = df[:101]
df.reset_index(drop = True, inplace = True)
df.head()

In [ ]:
#Function that will ask for the dollar size of the portfolio to build.
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
portfolio_input()

In [ ]:
#calculate the number of shares to buy and value of the position
position_size = float(portfolio_size) / len(df.index)
for i in range(0, len(df['Ticker'])):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / df['Price'][i])
    
df['Position Value'] = df['Number of Shares to Buy']*df['Price']
df.head()

In [ ]:
df.tail()

In [ ]:
df['Price']= df['Price'].astype(float).round(2)
df['Score']= df['Score'].astype(float).round(2)
df['Position Value']= df['Position Value'].astype(float).round(2)
for row in df.index:
    for period in periods:
        df[f'{period} Return Percentile'] = df[f'{period} Return Percentile'].astype(float).round(4)
        df[f'{period} Price Return'] = df[f'{period} Price Return'].astype(float).round(4)
df.head()

In [ ]:
#write the dataframe to an Excel file
writer = pd.ExcelWriter('momentum.xlsx', engine='xlsxwriter')
final_df = df
final_df.to_excel(writer, sheet_name='Momentum', index = False)
final_df.head()

In [ ]:
#formating for the Excel sheet
background_color = 'white'
font_color = 'black'


string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$#,#.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_format = writer.book.add_format(
        {
            'num_format':'#,#.00%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'#,#00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
#formate the Excel columns
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Company Name', string_format],
                    'C': ['One-Year Price Return', percent_format],
                    'D': ['One-Year Return Percentile', percent_format],
                    'E': ['Six-Month Price Return', percent_format],
                    'F': ['Six-Month Return Percentile', percent_format],
                    'G': ['Three-Month Price Return', percent_format],
                    'H': ['Three-Month Return Percentile', percent_format],
                    'I': ['One-Month Price Return', percent_format],
                    'J': ['One-Month Return Percentile', percent_format],
                    'K': ['Score', integer_format],
                    'L': ['Price', dollar_format],
                    'M': ['Number of Shares to Buy', integer_format],
                    'N': ['Position Value', dollar_format]
                    }

In [ ]:
for column in column_formats.keys():
    writer.sheets['Momentum'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()